# Interacting with the PYNQ Board

Before lunch, you started to get familiar with the Jupyter Notebook interface, writing Markdown, and running Python code. Nothing so far has been specific to the PYNQ board. This all could have easily been run on your laptop instead. Now, we'll start looking at the PYNQ board specifically — including playing with the on-board buttons, switches, and LED lights.

Before we start interacting with the physical world, let's try and convince ourselves that the code we write is indeed running on the PYNQ board.

## Contents

* [Getting PYNQ system information](#Getting-PYNQ-system-information)
* [LEDs, switches and buttons](#LEDs-switches-and-buttons)
* [Controlling an LED](#Controlling-an-LED)
* [Reading from switches and buttons](#Reading-from-switches-and-buttons)
* [Reacting to button presses](#Reacting-to-button-presses)
* [Summary](#Summary)


## Getting PYNQ system information

First of all, let's get the "hostname" of the device this code is running on. There is a system command (i.e. not a Python command) called `hostname` that does just this. Any command we type after a `!` will be executed as a system command. Try it below:

In [1]:
!hostname

pynq


The fact that the host name is "pynq" should start to convince you that this code is running on the board, and not your laptop!

Note that we can use the `!` symbol to run *any* Linux command we want. For example, we'll use this in some of the sessions tomorrow to install extra software.

Next, let's look at how much memory the device has:

In [2]:
!cat /proc/meminfo | grep 'MemTotal'

MemTotal:         508556 kB


So, we have about 500 MB (500 thousand kB). This is plenty for our needs, but your laptop probably has a lot more than this...

And finally, let's look at what type of CPU the device has:

In [3]:
!cat /proc/cpuinfo | grep name

model name	: ARMv7 Processor rev 0 (v7l)
model name	: ARMv7 Processor rev 0 (v7l)


This has two ARM CPUs. This type of CPU is much more likely to be found in a phone, raspberry pi (or a PYNQ board!) than a laptop.

We now should be happy that the commands we write are indeed run on the PYNQ board, so let's use it to interact with something physical!

## LEDs, switches and buttons

The PYNQ board some on-board controls, including:

  * Four green LEDs (called `LD0` to `LD3`)
  
  * Two multi-color LEDs (`LD4` and `LD5`)
  
  * Two slide-switches (`SW0` and `SW1`) and...
  
  * Four push-buttons (`BTN0` to `BTN3`)
  
Have a look at the board and try to identify each of these elements.

Note that there are additional push-buttons and LEDs on the board, but these are used for specific functions (Power LED, a reset button, etc.) and are not user accessible.

These inputs and outputs can be controlled directly from Python. To demonstrate this, we first need get a Python representation of the board — let's call it `board`.

In [4]:
from pynq.overlays.base import BaseOverlay

board = BaseOverlay("base.bit")

This `board` value has lots of different properties associated with it. The names for the buttons, switches, and LEDs are shown below:

![](./data/z2_peripherals.svg)

For example, if we wanted to do something with the rightmost LED, we would access it via `board.leds[0]`. Let's dive in to how to use each of these controls.

## Controlling an LED

Now that we have this `board` reference, we can start manipulating the LEDs. There are 3 main things we can do with each LED:

  1. Turn it on, using `.on()`
  2. Turn it off, using `.off()`
  3. "Toggle" it between off and on, using `.toggle()`

Let’s start by turning a single LED on and off.

In [5]:
led0 = board.leds[0]

In [6]:
led0.on()

Check the board and confirm the LED is on.

In [7]:
led0.off()

Let’s now toggle `led0` on and off automatically. We can use the `sleep()` method from the `time` package to introduce a delay. This should let us control how quickly the LED flashes.

For example, let's try toggling the LED 20 times using a `for` loop. We'll wait for 0.1 seconds each time before toggling the LED.

In [8]:
import time

for counter in range(20):
    led0.toggle()
    time.sleep(.1)

## Reading from switches and buttons

Next, we can start interacting with the LEDs using the switches and buttons. We can use `.read()` to get the value of a button or switch.

First, let's make sure that all of the LEDs off. Instead of writing 4 lines (one for each LED), let's use a `for` loop to turn off all the LEDs.

> **FOR REVIEW**: We could avoid doing any explicit indexing here if it would be easier... think `for led in board.leds:` and `for button, led in zip(board.buttons, board.leds):`. Thoughts?

In [9]:
for led_num in range(board.leds.length):
    board.leds[led_num].off()

Now let's read each button, and if the button is pressed down, the corresponding LED should be toggled. You can execute this cell a few times while pressing different buttons. Note that you'll need to hold these buttons down before clicking "Run".

In [10]:
for led_num in range(board.buttons.length):
    if board.buttons[led_num].read():
        board.leds[led_num].toggle()

Finally, let's use the switches to control the two multi-color LEDs.

Remember that these LEDs are numbers 4 and 5 (we must skip the 0, 1, 2, and 3). We will turn on each LED when the corresponding switch is "on". Because these are multi-color LEDs, we also need to tell it what color to use. Try changing the `color` number to different values between 1 and 7. 

In [11]:
color = 1 # Try changing this!

for switch_num in range(board.switches.length):
    led_num = switch_num + 4
    
    if board.switches[switch_num].read():
        board.rgbleds[led_num].on(color)
    else:
        board.rgbleds[led_num].off()

## Reacting to button presses

So far we have only read from the buttons once when we execute a cell. Because of this, you've probably been looking pretty silly trying to hold down buttons while also managing to click `Run`!

This is a little awkward, so what can we do about it? Let's start looking at how to dynamically react to button presses as soon as they happen. We'll start off with a simple technique called "polling" to help us check for button presses.

Previously, we used `.read()` to get immediate feedback from the button — either a 0 or a 1. To react to a button press, we'll use `.wait_for_value(...)`. This essentially says: stop what you're doing and *wait* until the button is pressed or released, depending on the number we supply. Let's see this in action by waiting for `BTN0` to be pressed, printing a message, and then waiting for `BTN0` to be released.

In [12]:
btn0 = board.buttons[0]

print('Waiting for BTN0 to be pressed...')

btn0.wait_for_value(1)
print('BTN0 has been pressed  😄')

btn0.wait_for_value(0)
print('BTN0 has been released 😔')

Waiting for BTN0 to be pressed...
BTN0 has been pressed  😄
BTN0 has been released 😔


We can put this code in a loop if we wanted to keep checking for more button presses. Let's try making this run forever, using the condition `while True`.

In [13]:
while True:
    btn0.wait_for_value(1)
    print('BTN0 has been pressed  😄')

    btn0.wait_for_value(0)
    print('BTN0 has been released 😔')

BTN0 has been pressed  😄
BTN0 has been released 😔
BTN0 has been pressed  😄
BTN0 has been released 😔


KeyboardInterrupt: 

We can now press and release the button many times and react to it with our code. We did tell Python to run this *forever* though... so this cell is still running! Only one cell can run at a time, so we will need to stop this before moving on. Stop this cell with the stop button (⏹️) in the toolbar.

And don't panic! When we stop the cell, there will be a big, red, scary error. This is just letting us know that something (us!) has interrupted the cell. Let's try to hide this message using Python's error handling.

We want to **try** running our loop forever, **except** when we get a `KeyboardInterrupt` event that comes from the stop button.

In [14]:
try:
    while True:
        btn0.wait_for_value(1)
        print('BTN0 has been pressed  😄')

        btn0.wait_for_value(0)
        print('BTN0 has been released 😔')
    
except KeyboardInterrupt:
    print ('Stopping the loop politely')

BTN0 has been pressed  😄
BTN0 has been released 😔
BTN0 has been pressed  😄
BTN0 has been released 😔
Stopping the loop politely


Neat! We've managed to hide the error message and exit the loop cleanly.

We've shown you a simple way to react to button events now. This approach will be enough for most needs, although there are a few restrictions:

  1. We can't do anything else in Python while we wait for a button to change value
  2. We can only wait for one button at a time

If you have time and are feeling brave, take a look at [a more complex example](./extra/ButtonsWithAsyncio.ipynb) that doesn't have these restrictions. This uses asynchronous programming, which can get a little mind-bending if you stare at it for too long.

## Summary

You've nearly gone through all of the training for day 1 now! Let's just summarize what we've covered:

  * Used system commands to get information specific to the PYNQ board
  * Started to interact with the physical world from Python!
    + Turning LEDs on and off
    + Reading input from switches
    + Reading input from push buttons
    + Reacting dynamically to button presses

This session will be running until 3:15, so feel free to try some of the following bonus challenges if you have time! 

### Bonus Challenges

Again, if you finish this session early, there are some optional challenges for you to work through. You now know enough to try some pretty neat things with the PYNQ board's inputs and outputs. For example, try:

  * Start by displaying a custom pattern on the 4 LEDs
  * Now, try making this pattern rotate (LD0 becomes LD1, LD1 becomes LD2, etc.) when a button is pressed
  * Let the direction of the rotation be set by one of the switches 
  * Try making an animation using the LEDs. Start simple, but get as complex as you'd like!
  * Have a look at [a more powerful way](./extra/ButtonsWithAsyncio.ipynb) of listening for button events (This is pretty advanced!)
  
----

[Back to the top](#Contents)

----